In [ ]:
# Mount Google drive on Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install timm

In [ ]:
!git clone https://github.com/philipjackson/style-augmentation.git

In [ ]:
!python style-augmentation/setup.py install

In [ ]:
!git clone https://github.com/jinczing/FixRes.git

In [ ]:
import torch
import numpy as np
import tqdm
import csv
import os
from timm.models import create_model
import torch.autograd.profiler as profiler
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from style_augmentation.styleaug import StyleAugmentor
from trainer import Trainer
from tester import Tester

In [ ]:
# Training(Transfer Learning) session. For inference, you can skip this block
trainer = Trainer(epoch=200, dataset_path='./drive/My Drive/datasets/car classification/train_data', 
                    val_path='./drive/My Drive/datasets/car classification/val_data', 
                    val_crop='five',
                    batch_size=4, 
                    model_name='tf_efficientnet_b6_ns', 
                    lr=0.0002, 
                    lr_min=0.0002, 
                    weight_decay=1.5e-4, 
                    momentum=0.9, 
                    log_step=200, 
                    save_step=1,
                    log_path='./drive/My Drive/cars_log.txt', 
                    cutout=True, 
                    style_aug=False,
                    resume=False,
                    resume_path='./drive/My Drive/ckpt/60.pth')
trainer.train()

In [ ]:
# FixEfficientNet Session. For inference, you can skip this block
!python FixRes/main_finetune_custom.py --architecture 'EfficientNet' --job-id 0 \
--data-path './drive/My Drive/datasets/car classification/' --input-size 680 --epochs 80 --batch 4\
--workers 1 --num-tasks 1 --learning-rate 1e-4 --EfficientNet-models 'tf_efficientnet_b6_ns' \
--shared-folder-path '/content/drive/My Drive/ckpt' --ckpt-path 'drive/My Drive/ckpt/63.pth' --classes 196

In [ ]:
# Inference Session
tester = Tester(epoch=1,
                dataset_path='./drive/My Drive/datasets/car classification/test_data', 
                batch_size=4, 
                model_name='tf_efficientnet_b6_ns', 
                ckpt_path='./drive/My Drive/ckpt/finetune/2/checkpoint_32.pth', 
                test_number=5000,
                pseudo_test=True, 
                crop='multi', 
                csv_path='./drive/My Drive/datasets/test.csv', 
                mode='fix', 
                sizes=(868, 768, 680, 600, 528))
tester.test()